<a href="https://colab.research.google.com/github/kyjhub/Web_Crawling/blob/main/naver_movie_all_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import pandas as pd
from urllib.parse import quote
from urllib.error import HTTPError, URLError

# 영화제목 입력

In [ ]:
movie_name_ = input('어떤 영화를 검색하시겠습니까 : ')
movie_name = quote(movie_name_)

In [ ]:
url = 'https://movie.naver.com/movie/search/result.naver?query=' + movie_name + '&section=all&ie=utf8'

In [ ]:
movie_page = requests.get(url).content
movie_page_soup = BeautifulSoup(movie_page, 'html.parser')
movie_lists = movie_page_soup.find('ul', {'class' : 'search_list_1'}).find_all('li')

num=1
movie_codes = []

for movie in movie_lists:

  href = movie.find('a').get('href')
  movie_code = href.split('=')[1]
  movie_codes.append(movie_code)

  print('===================' + str(num) + '번째 영화==================')
  print(movie.dl.get_text()[1:-3])
  num+=1

In [ ]:
n = int(input('몇번째 영화가 찾던 영화입니까?'))
search_movie_code = movie_codes[n-1]
print(search_movie_code)

## 전체평점 다 볼 수 있는 페이지 스크래핑

- 2167 리뷰에서 HTTPERROR 413 떠서 try except 넣어줌  
- 13번째 코드 url을 for문 밖에다가 뒀더니 16번째 코드 실행할 때 url1 -> url12 -> url123이렇게 떠서 범위를 벗어나서 http error 떴던것임 13번째코드를 지금처럼 for문 안에 넣었더니 해결됨

In [ ]:
movie_review_df = pd.DataFrame(columns=("Score", "Review"))

def get_movie_review(movie_code):
  num = 0
  url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=' + str(movie_code) + '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'  
    
  while True:
    try:
      print("페이지" + str(num) + " 리뷰")
      
      review_page = urllib.request.urlopen(url).read()
      review_page_soup = BeautifulSoup(review_page, 'html.parser')
      review_list = review_page_soup.select('body > div > div > div.score_result > ul > li')
      
      for idx, review_info in enumerate(review_list):
        score = review_info.find('em').get_text()
        review_text = review_info.find('span', {'id':'_filtered_ment_{}'.format(idx)}).get_text().strip()
        movie_review_df.loc[num] = [score, review_text]

      try:
        url = 'http://movie.naver.com' + review_page_soup.find('a', {'class' : 'pg_next'}).get('href')
        num += 1
      except:
        break

    except HTTPError as e:
      print(e)
      pass
  
  return movie_review_df

In [ ]:
review = get_movie_review(search_movie_code)
review

## 데이터프레임 csv파일로 저장

- encoding 안 정해줬더니 파일이 깨져서 encoding 설정

In [ ]:
review.to_csv('./sample_data/interstella_review.csv', encoding='utf-8-sig')

## csv파일 불러오기

In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
!ls

# 결측치 제거와 리뷰 데이터 개요 출력

In [ ]:
import pandas as pd
import numpy as np

review = pd.read_csv('./sample_data/interstella_review.csv')
#결측치 제거 전
before = review.shape[0]

review.dropna(inplace=True)

#결측치 제거 후
after = review.shape[0]

drop_n = before - after
print('결측치 제거 행 개수 = ' + str(drop_n))

print('반올림된 리뷰 평점 평균 : ' + str(review['Score'].mean().round()))

# 불용어 제거할 때 쉽게 하기위해 리스트로 변환
review_dict = review.to_dict()
review_val = review_dict['Review'].values()
reviews = list(review_val)

print('전체 리뷰 수 : ' + str(len(reviews)))

## 한국어 형태소 분석기 설치
- MeCab 설치

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
from konlpy.tag import Mecab
tagger = Mecab()

## 불용어 제거
- 각 영화 리뷰마다 불용어 리스트를 다르게 만들어야 적절하게 단어들 거를 수 있음
  - 인터스텔라 : 우주, 스텔라, 과학이라는 단어 많이 사용됨

In [ ]:
stop_words = '전 난 일 걸 뭐 줄 만 건 분 개 끝 잼 이거 번 중 듯 때 게 내 말 나 수 거 점 것 후 이 애 씨 속 뿐 밋 그 급 ㄷ 데'
stop_words = stop_words.split(' ')
print(stop_words)

In [ ]:
review_nouns = []

for review in reviews:
  for noun in tagger.nouns(review):
    if noun not in stop_words:
      review_nouns.append(noun)

review_nouns[:10]

## 단어 빈도수

In [ ]:
from collections import Counter

review_noun_count = Counter(review_nouns)
top_review_nouns = dict(review_noun_count.most_common(100))
top_review_nouns

## 한글 폰트 설정
- 그래프 폰트 깨짐 수정하기 위해

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 폰트 설정 (`이 코드 없으면 바차트 폰트 깨짐`)
- Google Colab의 노트북에서 위의 코드를 먼저 한번 실행 (폰트 설치 단계)
- 그 후 런타임 다시 시작 (설치된 폰트 적용 준비)
- 다시 위의 코드를 실행 (폰트 적용)
- 그 후 그래프 출력 코드 실행 (한글 정상 출력)
- 출처 : https://jonsyou.tistory.com/22
 

In [ ]:
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt

y_pos = np.arange(len(top_review_nouns))

plt.figure(figsize=(12, 24))
plt.barh(y_pos, top_review_nouns.values()) #수평 바차트
plt.title('Word Count')
plt.yticks(y_pos, top_review_nouns.keys())
plt.show();

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(background_color='white', font_path='./font/NanumBarunGothic.ttf')
wc.generate_from_frequencies(top_review_nouns)

In [ ]:
figure = plt.figure(figsize=(12, 12))
ax = figure.add_subplot(1, 1, 1)
ax.axis('off')
ax.imshow(wc)
plt.show()

- 다대일 
  - 입력이 여러개고 출력은 하나
  - 감정분석에 사용됨

- 언어를 숫자로 표현하는 것이 embedding

# 환경 준비
1. 라이브러리 다운로드
2. 네이버 영화평과 긍부정 데이터를 다운로드합니다 (파일보기 +  새로고침 후 확인)

* 자체 데이터셋을 사용할 경우 내용과 카테고리가 각각 content와 label 열에 들어가는 파일(아래 예시 참조)로 dataset.xlsx로 저장 후 기존 파일을 덮어쓰기 하면 됩니다. 
* 엑셀파일의 label과 content의 순서는 상관없으나 label은 0부터 시작하는 숫자로 입력하면 좋습니다. 예를들어 카테고리가 4개면 label을 0, 1, 2, 3으로 표시해주세요.

```
label  content
1      영화가 재밌다.     
1      이 영화 추천해요.     
0      지루한 영화였습니다.
...  
```

In [ ]:
!pip3 install -q transformers
!git clone https://github.com/kiyoungkim1/ReadyToUseAI

from ReadyToUseAI.src.nlp import make_sample_dataset, bert_sequence_classification
make_sample_dataset.nsmc(mode='test', text_only=False)  # mode: which datasets? 'train' or 'test'

# [Training] 
* 첨부된 샘플의 경우 약 40min 소요 (Tesla T4 GPU)
* min_sentence_length보다 긴 문장만 사용합니다.
* MAX_LEN은 모델이 인식하는 token의 길이로, 전체길이가 약 MAX_LEN의 2배보다 긴 문장은 뒷부분이 삭제됩니다 (예를들어 MAX_LEN = 128이면, 대략 길이가 256이상인 문장은 뒷부분이 무시됨).
* batch_size는 한번에 몇개의 sample을 계산하는지를 나타내며, 제한된 메모리에서 MAX_LEN을 줄이면 batch_size를 키울 수 있고, MAX_LEN를 키우면 batch_size를 줄여야 합니다. 
* epochs는 데이터셋을 몇번 반복해서 학습할지 여부이며, dataset_split은 전체 데이터 중 몇 %를 검증용 데이터셋으로 사용할지 여부입니다.

In [ ]:
CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=32, use_bert_tokenizer=True)
CLS.dataset(data_path='dataset.xlsx')
CLS.load_model(mode='train')
CLS.train(epochs=3, dataset_split=0.1)

# [Inference]
* sentences에 원하는 문장을 아래 형식과 같이 넣으면 해당하는 카테고리를 반환합니다.
* saved_model_path는 학습된 모델이 저장된 '폴더명'입니다.

In [ ]:
sentences = ['영화 재밌어요', '영화 재미없어요', '그냥 시간떼우기용', '완전 추천작']
saved_model_path='model/saved/3'

CLS = bert_sequence_classification.Classification(model_name='kykim/bert-kor-base', min_sentence_length=10, MAX_LEN=128, batch_size=64, use_bert_tokenizer=True)
CLS.load_model(mode='inference', saved_model_path=saved_model_path)
logit = CLS.inference(sentences=reviews[1000:2000])
print(logit)  # 네이버 영화평의 경우 0은 부정 카테고리, 1은 긍정 카테고리로 설정되어 있음